In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.utils import resample

import hyperopt

from catboost import CatBoostClassifier, Pool, cv

from sklearn.metrics import accuracy_score

In [3]:
# Read data
train = pd.read_csv("./data/clean_train.csv")
test = pd.read_csv("./data/clean_test.csv")
sample_submission = pd.read_csv("./data/sample_submission.csv")

train.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'type', 'stroke'],
      dtype='object')

In [4]:
def preprocessing(df):
    
    df.bmi = np.where(df.bmi.isnull(), np.where(df.age <= 4, 16,
                                               np.where(df.age <= 9, 16.5,
                                                       np.where(df.age <= 14, 18.5,
                                                               np.where(df.age == 15, 20, 23)))), df.bmi)
    
    
    df["df_glucheck"] = np.where((df.avg_glucose_level > 140) & (df.avg_glucose_level < 180), 
                          "HighSugarwithoutDiabetes", 
                                 np.where(df.avg_glucose_level >= 180, "HighSugarwithDiabetes",
                                         np.where((df.avg_glucose_level >= 95) & (df.avg_glucose_level <=140),"Fit","Normal" )))

    # Replacing smoking_status by unkown
    df['smoking_status'] = np.where(df['smoking_status'].isnull(), 'unknown', df['smoking_status'])
    df['age_g_67'] = np.where((df['age'] > 67) & (df['age'] < 73) , "Old" , np.where(df['age'] >= 73, "older", "young"))
    df['idealbmi'] = np.where((df['bmi'] > 19) & (df['bmi'] < 22.69), 1, 0)
    df['diabetesglucose'] = np.where(df['avg_glucose_level'] > 180, "Diabeticandhighsugar",
                                        np.where(df['avg_glucose_level'] > 140, "Highsugar","Normal"))
    df['age_l_30'] = np.where(df['age'] <=30,0,1)
#     df['femalewithdisease'] = np.where((df.gender == "Female") & (df.heart_disease == 1), 1, 0)
#     df['neversmokedfemale'] = np.where((df.gender == "Female") & (df.smoking_status == "never smoked"), 1, 0)
    
    df['privateandfit'] = np.where((df.heart_disease == 0) & (df.heart_disease == 0) & (df.work_type == "Private") & (df.ever_married == "Yes"), 1, 0)
    
    
    df = df.drop(['Residence_type','gender', 'type',  'avg_glucose_level'], axis = 1)
    df = pd.get_dummies(df)
    return df

In [5]:
train_v2 = preprocessing(train)
test_v2 = preprocessing(test)

In [6]:
train_v2['stroke'].value_counts()

0    42617
1      783
Name: stroke, dtype: int64

##### Up-sample Minority Class

In [7]:
# Separate majority and minority classes
df_majority = train_v2[train_v2.stroke==0]
df_minority = train_v2[train_v2.stroke==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=42617,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.stroke.value_counts()
# 1    576
# 0    576
# Name: balance, dtype: int64

1    42617
0    42617
Name: stroke, dtype: int64

In [8]:
train_x = df_upsampled.drop('stroke', axis = 1)
train_y = df_upsampled['stroke']

In [9]:
X_train, X_validation, y_train, y_validation = train_test_split(train_x, train_y, train_size=0.75, random_state=42)

X = train_x
y = train_y

/Users/sran12/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
from catboost import CatBoostClassifier
model=CatBoostClassifier(iterations=20000, depth=2  ,learning_rate=0.009689427269580251, loss_function='Logloss',od_type='Iter',
                            od_wait=20, use_best_model=True, l2_leaf_reg=3)
model.fit(X_train, y_train,eval_set=(X_validation, y_validation),plot=True)

0:	learn: 0.6891833	test: 0.6891667	best: 0.6891667 (0)	total: 79.8ms	remaining: 26m 36s
1:	learn: 0.6853990	test: 0.6853646	best: 0.6853646 (1)	total: 105ms	remaining: 17m 27s
2:	learn: 0.6816017	test: 0.6815518	best: 0.6815518 (2)	total: 143ms	remaining: 15m 54s
3:	learn: 0.6768948	test: 0.6767853	best: 0.6767853 (3)	total: 171ms	remaining: 14m 13s
4:	learn: 0.6723831	test: 0.6722166	best: 0.6722166 (4)	total: 193ms	remaining: 12m 50s
5:	learn: 0.6686925	test: 0.6684726	best: 0.6684726 (5)	total: 216ms	remaining: 11m 59s
6:	learn: 0.6650857	test: 0.6648134	best: 0.6648134 (6)	total: 239ms	remaining: 11m 21s
7:	learn: 0.6608662	test: 0.6605409	best: 0.6605409 (7)	total: 261ms	remaining: 10m 51s
8:	learn: 0.6570368	test: 0.6566474	best: 0.6566474 (8)	total: 282ms	remaining: 10m 25s
9:	learn: 0.6536835	test: 0.6532454	best: 0.6532454 (9)	total: 300ms	remaining: 10m
10:	learn: 0.6499927	test: 0.6494995	best: 0.6494995 (10)	total: 321ms	remaining: 9m 43s
11:	learn: 0.6467552	test: 0.64622

MetricVisualizer(data={'.': {'path': '.', 'name': 'current', 'content': {'passed_iterations': 53, 'total_iterations': 20000, 'data': {'iterations': [{'learn': [0.6891833182], 'iteration': 0, 'passed_time': 0.07981326378, 'remaining_time': 1596.185462, 'test': [0.689166749]}, {'learn': [0.6853990435], 'iteration': 1, 'passed_time': 0.1048053162, 'remaining_time': 1047.948356, 'test': [0.6853645998]}, {'learn': [0.6816017154], 'iteration': 2, 'passed_time': 0.1432650928, 'remaining_time': 954.9573539, 'test': [0.6815517514]}, {'learn': [0.6768947702], 'iteration': 3, 'passed_time': 0.1706677656, 'remaining_time': 853.1681601, 'test': [0.6767853136]}, {'learn': [0.6723830548], 'iteration': 4, 'passed_time': 0.1926547807, 'remaining_time': 770.4264682, 'test': [0.6722165657]}, {'learn': [0.6686925437], 'iteration': 5, 'passed_time': 0.2158161715, 'remaining_time': 719.171422, 'test': [0.6684726048]}, {'learn': [0.6650856629], 'iteration': 6, 'passed_time': 0.2386773027, 'remaining_time': 681.6964733, 'test': [0.6648134481]}, {'learn': [0.6608661648], 'iteration': 7, 'passed_time': 0.2605934118, 'remaining_time': 651.222936, 'test': [0.6605409039]}, {'learn': [0.6570367684], 'iteration': 8, 'passed_time': 0.2817145486, 'remaining_time': 625.7506157, 'test': [0.6566473549]}, {'learn': [0.6536834916], 'iteration': 9, 'passed_time': 0.3004230734, 'remaining_time': 600.5457236, 'test': [0.6532453587]}, {'learn': [0.6499927159], 'iteration': 10, 'passed_time': 0.3213356012, 'remaining_time': 583.925212, 'test': [0.6494995342]}, {'learn': [0.6467551883], 'iteration': 11, 'passed_time': 0.341333789, 'remaining_time': 568.5483145, 'test': [0.6462209888]}, {'learn': [0.6430291615], 'iteration': 12, 'passed_time': 0.3711220371, 'remaining_time': 570.5858582, 'test': [0.6424478456]}, {'learn': [0.6403047383], 'iteration': 13, 'passed_time': 0.3901279721, 'remaining_time': 556.9355465, 'test': [0.6397096185]}, {'learn': [0.6372746851], 'iteration': 14, 'passed_time': 0.4036855536, 'remaining_time': 537.8437193, 'test': [0.6366410775]}, {'learn': [0.634313837], 'iteration': 15, 'passed_time': 0.4174697669, 'remaining_time': 521.4197388, 'test': [0.6336426365]}, {'learn': [0.6315084485], 'iteration': 16, 'passed_time': 0.4329304807, 'remaining_time': 508.8970468, 'test': [0.6308305145]}, {'learn': [0.6281391661], 'iteration': 17, 'passed_time': 0.4442629969, 'remaining_time': 493.1812891, 'test': [0.6274185596]}, {'learn': [0.6253738701], 'iteration': 18, 'passed_time': 0.4557863391, 'remaining_time': 479.3193074, 'test': [0.624618155]}, {'learn': [0.6227046522], 'iteration': 19, 'passed_time': 0.4690760983, 'remaining_time': 468.6070222, 'test': [0.6219095448]}, {'learn': [0.6198201684], 'iteration': 20, 'passed_time': 0.4829182164, 'remaining_time': 459.4391926, 'test': [0.6189712212]}, {'learn': [0.6173655084], 'iteration': 21, 'passed_time': 0.4999061106, 'remaining_time': 453.9601944, 'test': [0.6165101511]}, {'learn': [0.6142239441], 'iteration': 22, 'passed_time': 0.5171045655, 'remaining_time': 449.1390394, 'test': [0.6133361863]}, {'learn': [0.6117911338], 'iteration': 23, 'passed_time': 0.5300524802, 'remaining_time': 441.1803477, 'test': [0.6108672236]}, {'learn': [0.6093833087], 'iteration': 24, 'passed_time': 0.5409248437, 'remaining_time': 432.1989501, 'test': [0.6084288455]}, {'learn': [0.6070306385], 'iteration': 25, 'passed_time': 0.5538909678, 'remaining_time': 425.5160843, 'test': [0.6060463286]}, {'learn': [0.6041753409], 'iteration': 26, 'passed_time': 0.5663800537, 'remaining_time': 418.9744005, 'test': [0.603155809]}, {'learn': [0.6022643268], 'iteration': 27, 'passed_time': 0.5791566192, 'remaining_time': 413.1041428, 'test': [0.6012429867]}, {'learn': [0.6000624909], 'iteration': 28, 'passed_time': 0.5923222199, 'remaining_time': 407.9057604, 'test': [0.5990130319]}, {'learn': [0.5979110919], 'iteration': 29, 'passed_time': 0.6043693744, 'remaining_time': 402.3085469, 'test': [0.5968341601]}, {'learn': [0.5960802459], 'i

54:	learn: 0.5553476	test: 0.5540400	best: 0.5540400 (54)	total: 973ms	remaining: 5m 52s
55:	learn: 0.5536248	test: 0.5523032	best: 0.5523032 (55)	total: 992ms	remaining: 5m 53s
56:	learn: 0.5524176	test: 0.5510987	best: 0.5510987 (56)	total: 1.01s	remaining: 5m 53s
57:	learn: 0.5513609	test: 0.5500407	best: 0.5500407 (57)	total: 1.03s	remaining: 5m 54s
58:	learn: 0.5500524	test: 0.5487220	best: 0.5487220 (58)	total: 1.05s	remaining: 5m 54s
59:	learn: 0.5487736	test: 0.5474331	best: 0.5474331 (59)	total: 1.07s	remaining: 5m 54s
60:	learn: 0.5477425	test: 0.5464021	best: 0.5464021 (60)	total: 1.09s	remaining: 5m 54s
61:	learn: 0.5461196	test: 0.5447713	best: 0.5447713 (61)	total: 1.1s	remaining: 5m 54s
62:	learn: 0.5450292	test: 0.5436864	best: 0.5436864 (62)	total: 1.12s	remaining: 5m 55s
63:	learn: 0.5440200	test: 0.5426785	best: 0.5426785 (63)	total: 1.14s	remaining: 5m 55s
64:	learn: 0.5431094	test: 0.5417526	best: 0.5417526 (64)	total: 1.16s	remaining: 5m 54s
65:	learn: 0.5416467	t

148:	learn: 0.4917990	test: 0.4900660	best: 0.4900660 (148)	total: 2.44s	remaining: 5m 25s
149:	learn: 0.4915644	test: 0.4898300	best: 0.4898300 (149)	total: 2.46s	remaining: 5m 25s
150:	learn: 0.4913095	test: 0.4895692	best: 0.4895692 (150)	total: 2.48s	remaining: 5m 25s
151:	learn: 0.4910812	test: 0.4893402	best: 0.4893402 (151)	total: 2.49s	remaining: 5m 25s
152:	learn: 0.4906433	test: 0.4889004	best: 0.4889004 (152)	total: 2.5s	remaining: 5m 24s
153:	learn: 0.4901650	test: 0.4884176	best: 0.4884176 (153)	total: 2.51s	remaining: 5m 24s
154:	learn: 0.4897795	test: 0.4880329	best: 0.4880329 (154)	total: 2.53s	remaining: 5m 23s
155:	learn: 0.4895034	test: 0.4877466	best: 0.4877466 (155)	total: 2.54s	remaining: 5m 23s
156:	learn: 0.4892702	test: 0.4875150	best: 0.4875150 (156)	total: 2.55s	remaining: 5m 22s
157:	learn: 0.4890602	test: 0.4873010	best: 0.4873010 (157)	total: 2.57s	remaining: 5m 22s
158:	learn: 0.4888289	test: 0.4870724	best: 0.4870724 (158)	total: 2.58s	remaining: 5m 21s


249:	learn: 0.4733997	test: 0.4714514	best: 0.4714514 (249)	total: 3.86s	remaining: 5m 4s
250:	learn: 0.4732551	test: 0.4713045	best: 0.4713045 (250)	total: 3.87s	remaining: 5m 4s
251:	learn: 0.4731886	test: 0.4712342	best: 0.4712342 (251)	total: 3.89s	remaining: 5m 4s
252:	learn: 0.4730442	test: 0.4710888	best: 0.4710888 (252)	total: 3.91s	remaining: 5m 4s
253:	learn: 0.4729795	test: 0.4710204	best: 0.4710204 (253)	total: 3.92s	remaining: 5m 4s
254:	learn: 0.4729076	test: 0.4709504	best: 0.4709504 (254)	total: 3.94s	remaining: 5m 4s
255:	learn: 0.4728371	test: 0.4708819	best: 0.4708819 (255)	total: 3.95s	remaining: 5m 4s
256:	learn: 0.4727742	test: 0.4708152	best: 0.4708152 (256)	total: 3.96s	remaining: 5m 4s
257:	learn: 0.4727126	test: 0.4707539	best: 0.4707539 (257)	total: 3.97s	remaining: 5m 4s
258:	learn: 0.4726512	test: 0.4706888	best: 0.4706888 (258)	total: 3.99s	remaining: 5m 3s
259:	learn: 0.4725934	test: 0.4706290	best: 0.4706290 (259)	total: 4s	remaining: 5m 3s
260:	learn: 0

...

19601:	learn: 0.3656008	test: 0.3671019	best: 0.3671019 (19601)	total: 4m 51s	remaining: 5.92s
19602:	learn: 0.3655986	test: 0.3671003	best: 0.3671003 (19602)	total: 4m 51s	remaining: 5.91s
19603:	learn: 0.3655954	test: 0.3670947	best: 0.3670947 (19603)	total: 4m 51s	remaining: 5.9s
19604:	learn: 0.3655927	test: 0.3670943	best: 0.3670943 (19604)	total: 4m 51s	remaining: 5.88s
19605:	learn: 0.3655898	test: 0.3670915	best: 0.3670915 (19605)	total: 4m 51s	remaining: 5.87s
19606:	learn: 0.3655881	test: 0.3670900	best: 0.3670900 (19606)	total: 4m 51s	remaining: 5.85s
19607:	learn: 0.3655856	test: 0.3670885	best: 0.3670885 (19607)	total: 4m 51s	remaining: 5.84s
19608:	learn: 0.3655831	test: 0.3670853	best: 0.3670853 (19608)	total: 4m 51s	remaining: 5.82s
19609:	learn: 0.3655806	test: 0.3670832	best: 0.3670832 (19609)	total: 4m 51s	remaining: 5.81s
19610:	learn: 0.3655783	test: 0.3670815	best: 0.3670815 (19610)	total: 4m 52s	remaining: 5.79s
19611:	learn: 0.3655766	test: 0.3670787	best: 0.367

19699:	learn: 0.3653618	test: 0.3668979	best: 0.3668979 (19699)	total: 4m 53s	remaining: 4.47s
19700:	learn: 0.3653595	test: 0.3668960	best: 0.3668960 (19700)	total: 4m 53s	remaining: 4.46s
19701:	learn: 0.3653591	test: 0.3668957	best: 0.3668957 (19701)	total: 4m 53s	remaining: 4.44s
19702:	learn: 0.3653568	test: 0.3668944	best: 0.3668944 (19702)	total: 4m 53s	remaining: 4.42s
19703:	learn: 0.3653552	test: 0.3668934	best: 0.3668934 (19703)	total: 4m 53s	remaining: 4.41s
19704:	learn: 0.3653549	test: 0.3668939	best: 0.3668934 (19703)	total: 4m 53s	remaining: 4.39s
19705:	learn: 0.3653541	test: 0.3668932	best: 0.3668932 (19705)	total: 4m 53s	remaining: 4.38s
19706:	learn: 0.3653457	test: 0.3668829	best: 0.3668829 (19706)	total: 4m 53s	remaining: 4.37s
19707:	learn: 0.3653441	test: 0.3668822	best: 0.3668822 (19707)	total: 4m 53s	remaining: 4.35s
19708:	learn: 0.3653428	test: 0.3668813	best: 0.3668813 (19708)	total: 4m 53s	remaining: 4.34s
19709:	learn: 0.3653408	test: 0.3668793	best: 0.36

19792:	learn: 0.3651607	test: 0.3667151	best: 0.3667151 (19792)	total: 4m 55s	remaining: 3.09s
19793:	learn: 0.3651585	test: 0.3667122	best: 0.3667122 (19793)	total: 4m 55s	remaining: 3.07s
19794:	learn: 0.3651569	test: 0.3667128	best: 0.3667122 (19793)	total: 4m 55s	remaining: 3.06s
19795:	learn: 0.3651551	test: 0.3667100	best: 0.3667100 (19795)	total: 4m 55s	remaining: 3.04s
19796:	learn: 0.3651524	test: 0.3667060	best: 0.3667060 (19796)	total: 4m 55s	remaining: 3.03s
19797:	learn: 0.3651507	test: 0.3667042	best: 0.3667042 (19797)	total: 4m 55s	remaining: 3.01s
19798:	learn: 0.3651476	test: 0.3667026	best: 0.3667026 (19798)	total: 4m 55s	remaining: 3s
19799:	learn: 0.3651462	test: 0.3667023	best: 0.3667023 (19799)	total: 4m 55s	remaining: 2.98s
19800:	learn: 0.3651427	test: 0.3666967	best: 0.3666967 (19800)	total: 4m 55s	remaining: 2.97s
19801:	learn: 0.3651404	test: 0.3666944	best: 0.3666944 (19801)	total: 4m 55s	remaining: 2.95s
19802:	learn: 0.3651391	test: 0.3666932	best: 0.36669

19880:	learn: 0.3649544	test: 0.3665182	best: 0.3665182 (19880)	total: 4m 56s	remaining: 1.78s
19881:	learn: 0.3649522	test: 0.3665174	best: 0.3665174 (19881)	total: 4m 56s	remaining: 1.76s
19882:	learn: 0.3649506	test: 0.3665144	best: 0.3665144 (19882)	total: 4m 57s	remaining: 1.75s
19883:	learn: 0.3649457	test: 0.3665117	best: 0.3665117 (19883)	total: 4m 57s	remaining: 1.73s
19884:	learn: 0.3649427	test: 0.3665085	best: 0.3665085 (19884)	total: 4m 57s	remaining: 1.72s
19885:	learn: 0.3649397	test: 0.3665042	best: 0.3665042 (19885)	total: 4m 57s	remaining: 1.7s
19886:	learn: 0.3649380	test: 0.3665020	best: 0.3665020 (19886)	total: 4m 57s	remaining: 1.69s
19887:	learn: 0.3649367	test: 0.3665010	best: 0.3665010 (19887)	total: 4m 57s	remaining: 1.67s
19888:	learn: 0.3649344	test: 0.3664993	best: 0.3664993 (19888)	total: 4m 57s	remaining: 1.66s
19889:	learn: 0.3649327	test: 0.3664975	best: 0.3664975 (19889)	total: 4m 57s	remaining: 1.64s
19890:	learn: 0.3649289	test: 0.3664963	best: 0.366

19967:	learn: 0.3647641	test: 0.3663401	best: 0.3663401 (19967)	total: 4m 58s	remaining: 479ms
19968:	learn: 0.3647557	test: 0.3663317	best: 0.3663317 (19968)	total: 4m 58s	remaining: 464ms
19969:	learn: 0.3647541	test: 0.3663299	best: 0.3663299 (19969)	total: 4m 58s	remaining: 449ms
19970:	learn: 0.3647520	test: 0.3663270	best: 0.3663270 (19970)	total: 4m 58s	remaining: 434ms
19971:	learn: 0.3647498	test: 0.3663245	best: 0.3663245 (19971)	total: 4m 58s	remaining: 419ms
19972:	learn: 0.3647478	test: 0.3663237	best: 0.3663237 (19972)	total: 4m 58s	remaining: 404ms
19973:	learn: 0.3647469	test: 0.3663239	best: 0.3663237 (19972)	total: 4m 58s	remaining: 389ms
19974:	learn: 0.3647448	test: 0.3663223	best: 0.3663223 (19974)	total: 4m 58s	remaining: 374ms
19975:	learn: 0.3647425	test: 0.3663198	best: 0.3663198 (19975)	total: 4m 58s	remaining: 359ms
19976:	learn: 0.3647385	test: 0.3663151	best: 0.3663151 (19976)	total: 4m 58s	remaining: 344ms
19977:	learn: 0.3647360	test: 0.3663123	best: 0.36

In [11]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_pred = model.predict(X_validation)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_validation, y_pred)
print(confusion_matrix)
print(f1_score(y_validation, y_pred, average="macro"))
print(precision_score(y_validation, y_pred, average="macro"))
print(recall_score(y_validation, y_pred, average="macro"))    

[[8442 2239]
 [ 952 9676]]
0.849747188008593
0.8553721623546819
0.8504003623471195


In [12]:
y_pred_t = model.predict(test_v2)
y_pred_t.sum()

4171.0

In [13]:
sample_submission.stroke = y_pred_t
sample_submission.to_csv("./data/output/6catboost.csv", index=False)
